In [4]:
# --- 🔹 RAG sobre tu dataset de música (music_data.json) 🔹 ---

import json
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import textwrap

# -------------------------------------------------------------
# 1️⃣ Cargar los datos embebidos desde tu JSON
# -------------------------------------------------------------
with open("music_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"✅ {len(data)} documentos cargados.")

# -------------------------------------------------------------
# 2️⃣ Preparar los embeddings y textos
# -------------------------------------------------------------
texts = [d["text"] for d in data]
embeddings = np.array([d["embedding"] for d in data])

# -------------------------------------------------------------
# 3️⃣ Cargar el modelo de embeddings (igual que en tu scraper)
# -------------------------------------------------------------
model = SentenceTransformer("all-MiniLM-L6-v2")

# -------------------------------------------------------------
# 4️⃣ Función para buscar por similitud
# -------------------------------------------------------------
def buscar_mas_relevantes(pregunta, top_k=3):
    q_emb = model.encode([pregunta])
    sims = cosine_similarity(q_emb, embeddings)[0]
    indices = np.argsort(sims)[::-1][:top_k]
    resultados = [(texts[i], sims[i], data[i]) for i in indices]
    return resultados

# -------------------------------------------------------------
# 5️⃣ Generador de respuesta simple (versión básica sin API externa)
# -------------------------------------------------------------
def responder(pregunta, top_k=3):
    resultados = buscar_mas_relevantes(pregunta, top_k=top_k)

    print("\n📘 Pregunta:", pregunta)
    print("\n🔎 Documentos más relevantes:\n")

    for i, (texto, sim, doc) in enumerate(resultados, 1):
        print(f"--- [{i}] ({sim:.2f}) {doc['title']} — {doc['artist']}")
        wrapped = textwrap.fill(doc['text'], width=90)
        print(f"{wrapped}\n")

    # 🧠 Respuesta generada (versión sencilla)
    respuesta = (
        "Según los datos encontrados, parece que la respuesta está relacionada con: "
        + ", ".join([doc["title"] for _, _, doc in resultados if doc["title"]])
        + "."
    )

    print("💬 Respuesta generada:\n", respuesta)

# -------------------------------------------------------------
# 6️⃣ Ejemplo de uso
# -------------------------------------------------------------
# Puedes cambiar la pregunta por lo que quieras:
responder("¿Qué álbumes metal?")


✅ 50 documentos cargados.

📘 Pregunta: ¿Qué álbumes metal?

🔎 Documentos más relevantes:

--- [1] (0.48) Latin-Soul-Rock — 
Latin-Soul-Rock

--- [2] (0.45) Scheisse Nicht Schon Wieder Bernstein — Kommando Sonne-nmilch
Scheisse Nicht Schon Wieder Bernstein | Kommando Sonne-nmilch | Rock | Punk | Germany |
Vinyl LP Album Limited Edition | Major Label (2)

--- [3] (0.44) Caer — Twin Shadow
Caer | Twin Shadow | Electronic, Rock, Pop | Synth-pop | US | Vinyl LP Album Limited
Edition; Flexi-disc 7" 45 RPM Single Limited Edition | Warner Bros. Records

💬 Respuesta generada:
 Según los datos encontrados, parece que la respuesta está relacionada con: Latin-Soul-Rock, Scheisse Nicht Schon Wieder Bernstein, Caer.
